In [0]:
from pyspark.sql.functions import *

####Notebook Parameterization using widgest

In [0]:
dbutils.widgets.text("File_name","")
p_file_name=dbutils.widgets.get("File_name")

####Data Reading

In [0]:
df=spark.readStream.format("cloudFiles")\
  .option('cloudFiles.format',"parquet")\
  .option("cloudFiles.schemaLocation",f"abfss://bronze@databrickseteprojectsa.dfs.core.windows.net/checkpoint_{p_file_name}")\
    .load(f"abfss://source@databrickseteprojectsa.dfs.core.windows.net/{p_file_name}")

###Add metadata columns [ingest_ts,source_file]

In [0]:
df=df.withColumn("ingest_ts",current_timestamp())\
    .withColumn('source_file',col("_metadata.file_path"))

####Data Writing

In [0]:
q=df.writeStream.format("parquet").outputMode("append")\
  .option("checkpointLocation",f"abfss://bronze@databrickseteprojectsa.dfs.core.windows.net/checkpoint_{p_file_name}")\
    .option("path",f"abfss://bronze@databrickseteprojectsa.dfs.core.windows.net/{p_file_name}")\
      .trigger(once=True)\
        .start()
q.awaitTermination()


In [0]:
source_path=f'abfss://source@databrickseteprojectsa.dfs.core.windows.net/{p_file_name}'
bronze_path=f'abfss://bronze@databrickseteprojectsa.dfs.core.windows.net/{p_file_name}'

In [0]:
df_source=spark.read.format('parquet').load(source_path).count()
print(f"source count :{df_source}")
df_bronze=spark.read.format('parquet').load(bronze_path).count()
print(f"bronze count:{df_bronze}")

source count :9554778
bronze count:9554778
